# Final project - Battle of Neighborhood
## Project Introduction
Hanoi is the capital of Hanoi. Mr Long is planning to open a restaturant. He is contemplating on the  Vietnamese Food restaurant, and where to locate it. Therefore, he use Foursquare API to find clusters of venues in 9 urban districts of
Hanoi.
## Data description
The csv file contains name and geo coordinates of Hanoi. He will read it into pandas data frame and query 100 venues in radius of 500m around each district. And then he will find 5 cluster of venues. To minimize competition, he will choose the cluster with no Vietnmese Food restaurant in top 10 common venues.

## Implementation

### Define Foursquare credentials, make request to 100 venues in Hanoi

In [ ]:
CLIENT_ID = 'C1YS0EU4C0MTTCNDINPNXBMFDM2YPBBEZ1VQL5IFWLIXU4EC' # your Foursquare ID
CLIENT_SECRET = 'AWGGKRYPEFT24D2NPAD2MD3VREK5GMVJFXNM3ZZD25JARGN0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [22]:
lattitude, longitude = 21.028511, 105.804817
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lattitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=C1YS0EU4C0MTTCNDINPNXBMFDM2YPBBEZ1VQL5IFWLIXU4EC&client_secret=AWGGKRYPEFT24D2NPAD2MD3VREK5GMVJFXNM3ZZD25JARGN0&v=20180605&ll=21.028511,105.804817&radius=5000&limit=500'

In [5]:
import requests

In [6]:
import json

In [23]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5baf3b594c1f6750e0699e26'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Hanoi',
  'headerFullLocation': 'Hanoi',
  'headerLocationGranularity': 'city',
  'totalResults': 237,
  'suggestedBounds': {'ne': {'lat': 21.073511045000046,
    'lng': 105.85293777280498},
   'sw': {'lat': 20.983510954999957, 'lng': 105.75669622719502}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '55270376498e18fba3a70435',
       'name': 'Lotte Hotel Hanoi',
       'location': {'address': '54 Lieu Giai',
        'lat': 21.032241317359276,
        'lng': 105.81281730819838,
        'labeledLatLngs': [{'label': 'display',
          'lat': 21.032241317359276,
       

In [19]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [64]:
venues = results['response']['groups'][0]['items']
import json
from pandas.io.json import json_normalize
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Lotte Hotel Hanoi,Hotel,21.032241,105.812817
1,Ofukuro-Tei おふくろ亭,Japanese Restaurant,21.028523,105.806315
2,Công Viên Cầu Giấy (Yên Hoà),Park,21.029339,105.789912
3,Café Cuối Ngõ,Coffee Shop,21.033250,105.802597
4,CGV Cinemas Vincom Nguyễn Chí Thanh,Multiplex,21.023058,105.809507


In [25]:
nearby_venues.shape

(100, 4)

In [12]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


### Load csv file and get nearby values of each district (Neighborhood)

In [26]:
import pandas as pd

In [66]:
hanoi = pd.read_csv('hanoi.csv')

In [67]:
hanoi.columns = ['Neighborhood', 'Lat', 'Long']

In [68]:
hanoi.head()

,Neighborhood,Lat,Long
0,Hoan Kiem,21.02828,105.85388
1,Hai Ba Trung,21.01001,105.84980
2,Ba Dinh,21.03403,105.81426
3,Hoang Mai,20.97455,105.84436
4,Thanh Xuan,20.99404,105.80088


In [69]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [71]:
hanoi_venues = getNearbyVenues(names=hanoi['Neighborhood'],
                                   latitudes=hanoi['Lat'],
                                   longitudes=hanoi['Long']
                                  )

Hoan Kiem
Hai Ba Trung
Ba Dinh
Hoang Mai
Thanh Xuan
Cau Giay
Dong Da
Ha Dong
Tay Ho
Long Bien


In [72]:
hanoi_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hoan Kiem,21.02828,105.85388,Apricot Boutique Hotel,21.028239,105.851055,Hotel
1,Hoan Kiem,21.02828,105.85388,Pizza 4P,21.025141,105.854438,Italian Restaurant
2,Hoan Kiem,21.02828,105.85388,Sofitel Legend Metropole Hanoi,21.025527,105.855960,Hotel
3,Hoan Kiem,21.02828,105.85388,Hoa 10 giờ Floral & Book Café,21.030677,105.856144,Café
4,Hoan Kiem,21.02828,105.85388,Vintage 1976,21.025981,105.853402,Café
5,Hoan Kiem,21.02828,105.85388,Starbucks Reserve Nhà Thờ,21.029162,105.850222,Coffee Shop
6,Hoan Kiem,21.02828,105.85388,Hanoi La Selva Hotel,21.031374,105.854886,Hotel
7,Hoan Kiem,21.02828,105.85388,Le Beaulieu,21.026041,105.855637,French Restaurant
8,Hoan Kiem,21.02828,105.85388,Hanoi Elegance Diamond,21.031546,105.855035,Hotel
9,Hoan Kiem,21.02828,105.85388,Bamboo Bar,21.025566,105.855958,Hotel Bar


In [73]:
hanoi_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Ba Dinh,26,26,26,26,26,26
Cau Giay,4,4,4,4,4,4
Dong Da,10,10,10,10,10,10
Ha Dong,4,4,4,4,4,4
Hai Ba Trung,50,50,50,50,50,50
Hoan Kiem,100,100,100,100,100,100
Hoang Mai,6,6,6,6,6,6
Long Bien,5,5,5,5,5,5
Tay Ho,6,6,6,6,6,6


In [74]:
print('There are {} uniques categories.'.format(len(hanoi_venues['Venue Category'].unique())))

There are 64 uniques categories.


In [75]:
hanoi_onehot = pd.get_dummies(hanoi_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hanoi_onehot['Neighborhood'] = hanoi_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [hanoi_onehot.columns[-1]] + list(hanoi_onehot.columns[:-1])
hanoi_onehot = hanoi_onehot[fixed_columns]

hanoi_onehot.head()

,Neighborhood,Arepa Restaurant,Australian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Bubble Tea Shop,Bus Station,Café,Coffee Shop,...,Sushi Restaurant,Tea Room,Temple,Tennis Stadium,Thai Restaurant,Tourist Information Center,Train Station,Ukrainian Restaurant,Vietnamese Restaurant,Wings Joint
0,Hoan Kiem,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hoan Kiem,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hoan Kiem,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hoan Kiem,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Hoan Kiem,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
hanoi_grouped = hanoi_onehot.groupby('Neighborhood').mean().reset_index()
hanoi_grouped

,Neighborhood,Arepa Restaurant,Australian Restaurant,BBQ Joint,Bakery,Breakfast Spot,Bubble Tea Shop,Bus Station,Café,Coffee Shop,...,Sushi Restaurant,Tea Room,Temple,Tennis Stadium,Thai Restaurant,Tourist Information Center,Train Station,Ukrainian Restaurant,Vietnamese Restaurant,Wings Joint
0,Ba Dinh,0.00,0.00,0.00,0.00,0.000000,0.00,0.0,0.230769,0.00,...,0.038462,0.038462,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.038462,0.038462
1,Cau Giay,0.00,0.00,0.00,0.00,0.000000,0.00,0.0,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.250000,0.000000
2,Dong Da,0.00,0.00,0.00,0.00,0.000000,0.00,0.0,0.200000,0.20,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Ha Dong,0.00,0.00,0.00,0.25,0.000000,0.00,0.0,0.250000,0.00,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
4,Hai Ba Trung,0.02,0.00,0.02,0.02,0.000000,0.02,0.0,0.080000,0.16,...,0.000000,0.020000,0.00,0.000000,0.02,0.000000,0.000000,0.000000,0.140000,0.000000
5,Hoan Kiem,0.00,0.01,0.00,0.01,0.000000,0.00,0.0,0.120000,0.14,...,0.030000,0.010000,0.01,0.000000,0.00,0.000000,0.000000,0.000000,0.100000,0.000000
6,Hoang Mai,0.00,0.00,0.00,0.00,0.000000,0.00,0.0,0.500000,0.00,...,0.000000,0.000000,0.00,0.000000,0.00,0.166667,0.166667,0.000000,0.000000,0.000000
7,Long Bien,0.00,0.00,0.00,0.00,0.000000,0.00,0.2,0.200000,0.00,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.200000,0.000000,0.000000,0.000000
8,Tay Ho,0.00,0.00,0.00,0.00,0.000000,0.00,0.0,0.166667,0.00,...,0.000000,0.000000,0.00,0.166667,0.00,0.000000,0.000000,0.166667,0.000000,0.000000
9,Thanh Xuan,0.00,0.00,0.00,0.00,0.333333,0.00,0.0,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


In [77]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Get top 10 venues of each neighborhood

In [78]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = hanoi_grouped['Neighborhood']

for ind in np.arange(hanoi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(hanoi_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Ba Dinh,Café,Hotel,Japanese Restaurant,Korean Restaurant,Steakhouse,Wings Joint,Vietnamese Restaurant,Mobile Phone Shop,Dim Sum Restaurant,Roof Deck
1,Cau Giay,Vietnamese Restaurant,Japanese Restaurant,Mobile Phone Shop,Park,Food Truck,Italian Restaurant,Indie Movie Theater,Ice Cream Shop,Hotpot Restaurant,Hotel Bar
2,Dong Da,Coffee Shop,Café,Movie Theater,Garden,Park,Dessert Shop,Lake,Hotpot Restaurant,Ice Cream Shop,French Restaurant
3,Ha Dong,Café,Market,Bakery,Hotel,Hotel Bar,Frozen Yogurt Shop,Garden,Grocery Store,Halal Restaurant,Wings Joint
4,Hai Ba Trung,Coffee Shop,Japanese Restaurant,Vietnamese Restaurant,Café,Fast Food Restaurant,Dessert Shop,Italian Restaurant,Noodle House,Pizza Place,Market
5,Hoan Kiem,Hotel,Coffee Shop,Café,Vietnamese Restaurant,Italian Restaurant,Ice Cream Shop,Sushi Restaurant,Noodle House,Hotel Bar,Fast Food Restaurant
6,Hoang Mai,Café,Train Station,Tourist Information Center,Hotpot Restaurant,Wings Joint,Hotel,French Restaurant,Frozen Yogurt Shop,Garden,Grocery Store
7,Long Bien,Café,Train Station,Electronics Store,Grocery Store,Bus Station,Hotel Bar,Frozen Yogurt Shop,Garden,Halal Restaurant,Hotel
8,Tay Ho,Lake,Hotel,Ukrainian Restaurant,Tennis Stadium,Mexican Restaurant,Café,French Restaurant,Frozen Yogurt Shop,Garden,Grocery Store
9,Thanh Xuan,Food Truck,Breakfast Spot,Mobile Phone Shop,Wings Joint,Japanese Restaurant,Italian Restaurant,Indie Movie Theater,Ice Cream Shop,Hotpot Restaurant,Hotel Bar


### Cluster neighborhood

In [79]:
from sklearn.cluster import KMeans
kclusters = 5

hanoi_grouped_clustering = hanoi_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hanoi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 4, 1, 1, 1, 1, 3, 2, 1, 0])

In [80]:
hanoi.head()

,Neighborhood,Lat,Long
0,Hoan Kiem,21.02828,105.85388
1,Hai Ba Trung,21.01001,105.84980
2,Ba Dinh,21.03403,105.81426
3,Hoang Mai,20.97455,105.84436
4,Thanh Xuan,20.99404,105.80088


In [81]:
hanoi_merged = hanoi

# add clustering labels
hanoi_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
hanoi_merged = hanoi_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

hanoi_merged # check the last columns!

,Neighborhood,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Hoan Kiem,21.02828,105.85388,1,Hotel,Coffee Shop,Café,Vietnamese Restaurant,Italian Restaurant,Ice Cream Shop,Sushi Restaurant,Noodle House,Hotel Bar,Fast Food Restaurant
1,Hai Ba Trung,21.01001,105.84980,4,Coffee Shop,Japanese Restaurant,Vietnamese Restaurant,Café,Fast Food Restaurant,Dessert Shop,Italian Restaurant,Noodle House,Pizza Place,Market
2,Ba Dinh,21.03403,105.81426,1,Café,Hotel,Japanese Restaurant,Korean Restaurant,Steakhouse,Wings Joint,Vietnamese Restaurant,Mobile Phone Shop,Dim Sum Restaurant,Roof Deck
3,Hoang Mai,20.97455,105.84436,1,Café,Train Station,Tourist Information Center,Hotpot Restaurant,Wings Joint,Hotel,French Restaurant,Frozen Yogurt Shop,Garden,Grocery Store
4,Thanh Xuan,20.99404,105.80088,1,Food Truck,Breakfast Spot,Mobile Phone Shop,Wings Joint,Japanese Restaurant,Italian Restaurant,Indie Movie Theater,Ice Cream Shop,Hotpot Restaurant,Hotel Bar
5,Cau Giay,21.02744,105.79026,1,Vietnamese Restaurant,Japanese Restaurant,Mobile Phone Shop,Park,Food Truck,Italian Restaurant,Indie Movie Theater,Ice Cream Shop,Hotpot Restaurant,Hotel Bar
6,Dong Da,21.01348,105.82679,3,Coffee Shop,Café,Movie Theater,Garden,Park,Dessert Shop,Lake,Hotpot Restaurant,Ice Cream Shop,French Restaurant
7,Ha Dong,20.97136,105.77876,2,Café,Market,Bakery,Hotel,Hotel Bar,Frozen Yogurt Shop,Garden,Grocery Store,Halal Restaurant,Wings Joint
8,Tay Ho,21.06632,105.81972,1,Lake,Hotel,Ukrainian Restaurant,Tennis Stadium,Mexican Restaurant,Café,French Restaurant,Frozen Yogurt Shop,Garden,Grocery Store
9,Long Bien,21.04902,105.88288,0,Café,Train Station,Electronics Store,Grocery Store,Bus Station,Hotel Bar,Frozen Yogurt Shop,Garden,Halal Restaurant,Hotel


### Mapping clusters using folium

In [53]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [54]:
import folium

In [59]:
latitude  = 21.028511
longitude =  105.804817

In [98]:
# create map
map_clusters = folium.Map(location=[lattitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(hanoi_merged['Lat'], hanoi_merged['Long'], hanoi_merged['Neighborhood'], hanoi_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining clusters

In [88]:
hanoi_merged.loc[hanoi_merged['Cluster Labels'] == 0, hanoi_merged.columns[[0] + list(range(3, hanoi_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Long Bien,0,Café,Train Station,Electronics Store,Grocery Store,Bus Station,Hotel Bar,Frozen Yogurt Shop,Garden,Halal Restaurant,Hotel


In [89]:
hanoi_merged.loc[hanoi_merged['Cluster Labels'] == 1, hanoi_merged.columns[[0] + list(range(3, hanoi_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Hoan Kiem,1,Hotel,Coffee Shop,Café,Vietnamese Restaurant,Italian Restaurant,Ice Cream Shop,Sushi Restaurant,Noodle House,Hotel Bar,Fast Food Restaurant
2,Ba Dinh,1,Café,Hotel,Japanese Restaurant,Korean Restaurant,Steakhouse,Wings Joint,Vietnamese Restaurant,Mobile Phone Shop,Dim Sum Restaurant,Roof Deck
3,Hoang Mai,1,Café,Train Station,Tourist Information Center,Hotpot Restaurant,Wings Joint,Hotel,French Restaurant,Frozen Yogurt Shop,Garden,Grocery Store
4,Thanh Xuan,1,Food Truck,Breakfast Spot,Mobile Phone Shop,Wings Joint,Japanese Restaurant,Italian Restaurant,Indie Movie Theater,Ice Cream Shop,Hotpot Restaurant,Hotel Bar
5,Cau Giay,1,Vietnamese Restaurant,Japanese Restaurant,Mobile Phone Shop,Park,Food Truck,Italian Restaurant,Indie Movie Theater,Ice Cream Shop,Hotpot Restaurant,Hotel Bar
8,Tay Ho,1,Lake,Hotel,Ukrainian Restaurant,Tennis Stadium,Mexican Restaurant,Café,French Restaurant,Frozen Yogurt Shop,Garden,Grocery Store


In [90]:
hanoi_merged.loc[hanoi_merged['Cluster Labels'] == 2, hanoi_merged.columns[[0] + list(range(3, hanoi_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Ha Dong,2,Café,Market,Bakery,Hotel,Hotel Bar,Frozen Yogurt Shop,Garden,Grocery Store,Halal Restaurant,Wings Joint


In [91]:
hanoi_merged.loc[hanoi_merged['Cluster Labels'] == 3, hanoi_merged.columns[[0] + list(range(3, hanoi_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Dong Da,3,Coffee Shop,Café,Movie Theater,Garden,Park,Dessert Shop,Lake,Hotpot Restaurant,Ice Cream Shop,French Restaurant


In [92]:
hanoi_merged.loc[hanoi_merged['Cluster Labels'] == 4, hanoi_merged.columns[[0] + list(range(3, hanoi_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Hai Ba Trung,4,Coffee Shop,Japanese Restaurant,Vietnamese Restaurant,Café,Fast Food Restaurant,Dessert Shop,Italian Restaurant,Noodle House,Pizza Place,Market


## Conclusion

It can be clearly seen that Cluster 1 is crowded with restaurants, including Vietnamese Foods restaurant. He would choose Long Bien (Cluster 0), Ha Dong (Cluster 3), or Hai Ba Trung (Cluster 4) with no Vietnamse Foods restaurent in top 10 venues. The final venues depends on the hiring price and location of hiring venue in these districts.